In [14]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF 벡터화용
from sklearn.metrics.pairwise import cosine_similarity # 유사도 분석용

In [15]:
# 카테고리 받기 전 결측치 전처리
def recipe_preprocess(recipe):
    
    # '소스' 열 결측치 '-' 로 대체
    recipe['sorce'].fillna('-', inplace=True)
    
    # 전처리 데이터 중 '재료' 열에 NaN 이 있는 경우 해당 행 모두 삭제
    recipe.dropna(axis=0,inplace=True,subset=['ingredient'])

In [16]:
# 분류 값 입력
def get_category():
    
    # 카테고리 선택 범위
    cat_list = ['빵','메인반찬','찌개','국 탕','디저트','밑반찬','양식','퓨전']
    print(f'현재 선택할 수 있는 카테고리는 {cat_list} 입니다.')
    
    while(True):
        cat = input('찾고 싶은 레시피의 분류를 입력해주세요 : ')
        
        # 카테고리 선택 범위 밖에 있는 경우 무한 루프를 돌게 됨
        if cat not in cat_list:
            print('카테고리에 있는 값이 아닙니다. 다시 입력해주세요')
        else:
            break
    return cat

In [17]:
# 카테고리 선택 후 필터링
def cat_preprocess(cat, recipe):
    
    # 1.기존 recipe DF에서 cooking_type열이 cat과 같은 데이터를 DF화 
    recipe1 = recipe[(recipe['cooking_type'] == cat)]
    
    # 기존 인덱스 번호 초기화
    recipe1.reset_index(inplace=True)
    
    # 2. 1의 DF에서 재료만 리스트 형태로 반환
    origin_recipe = recipe1['ingredient'].tolist()
    
    # 인덱스를 통해 자료를 뽑기 위한 recipe1 과 vectorizer 추가를 위한 리스트 형태의 origin_recipe 반환
    return recipe1, origin_recipe

In [33]:
# 재료 입력
def get_ingredients():
    cnt = 1
    recipe_list = []
    ingredient = ''
    print('최대 5개까지의 재료를 입력해주시고, 입력이 마무리 된 경우 x 입력해주세요')
    while(cnt < 6):
        ingredient_element = input('재료를 입력해주세요. : ')
                                   
        #재료입력에 대소문자 'X' 혹은 공백이 들어올 경우 입력이 마무리 됨
        if ingredient_element == 'x' or ingredient_element=='X' or ingredient_element ==' ': 
            break
        else:
            recipe_list.append(ingredient_element)
                                   
        cnt += 1
                                   
    print(f'선택된 재료들은 {recipe_list} 입니다.')
    input_recipe = ','.join(recipe_list)
    return input_recipe

In [19]:
# 벡터화
def recipe_vectorizer(input_recipe, origin_recipe):
    
    # input_recipe = 입력받은 레시피 리스트의 str 형태, original_recipe = 레시피 리스트 형태
    origin_recipe.append(input_recipe)
    
    # TfidfVectorizer() 객체 변수 생성, 깡통 모델
    vectorizer = TfidfVectorizer()
    
    # recipe 재료 데이터 벡터화, .todense 는 반환 값 보고싶을 때 사용(사용 안 할경우 성능 향상 있음)
    vec = vectorizer.fit_transform(origin_recipe)
    
    return vec

In [20]:
# 벡터화된 레시피 값을 받아 유사도 분석 후 분석된 레시피의 기존 index 값과 유사도 상위 5개를 DataFrame 반환
def sim_indexPreprocess (recipe_vec):
    
    # 입력된 recipe와 전체 recipe 의 유사도분석
    sim = pd.DataFrame(cosine_similarity(recipe_vec[-1] , recipe_vec))
    
    # 유사도분석 된 자료를 정렬 후 상위 5개까지 추출 정렬
    sim.sort_values(by=0,axis=1,ascending=False,inplace=True)
    
    # 유사도 분석 df와 레시피 df의 인덱스 행을 맞추기 위해 행열 전환처리(T=transpose) + 0번을 제외한 유사도 상위 5개 반환
    sim_per = sim.iloc[ : , 1:6].T
    
    # 컬렴명(= recipe index 번호 추출)
    recipe_index = sim_per.index.tolist()
    
    return recipe_index, sim_per

In [21]:
# 유사도 결과 출력
def recommendation_result(index_list, ref_recipe):
    print('입력하신 재료와 유사한 레시피 최대 5개를 추출합니다.\n')

    for info in index_list:
        title = ref_recipe.iloc[info]['title'] #제목
        ingredient = ref_recipe.iloc[info]['ingredient'] #재료
        url = ref_recipe.iloc[info]['uri'] # URL
        source = ref_recipe.iloc[info]['sorce'] # 소스
        similarity = (sim_per.loc[info][0])*100 # 유사도
        print(f'레시피 : {title} (유사도:{similarity:.1f}%)')
        print(f'재  료 : {ingredient}')
        print(f' url  : {url}')
        print(f'소  스 : {source}')
        print('\n')

In [22]:
recipe = pd.read_csv('recipe2.csv', encoding='utf-8')
recipe.head()

,idx,page_num,seq,cooking_type,title,ingredient,sorce,uri,insert_dt
0,1,1,1,밑반찬,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,"새송이버섯,버터,어린잎채소","간장,맛술,올리고당",https://www.10000recipe.com/recipe/6912734,2021-10-06 18:23:56
1,2,1,2,밑반찬,500원 이면 밥한끼~ 뚝딱하는 팽이버섯조림~ 너무 맛있어용^^~,"팽이버섯,청양고추,식용유,다진마늘","고추가루,설탕,미림,진간장,굴소스,고추장,된장,물",https://www.10000recipe.com/recipe/6956266,2021-10-06 18:23:57
2,3,1,3,밑반찬,[마늘종볶음]단짠단짠 자꾸만 손이가요~,"건새우,물,마늘종,소금","설탕,간장,식용유,참기름,물엿,통깨",https://www.10000recipe.com/recipe/6957535,2021-10-06 18:23:57
3,4,1,4,밑반찬,달콤 짭조름♡ 떡 베이컨 간장조림♡,"베이컨,대파,떡","간장,참기름,물엿,기름",https://www.10000recipe.com/recipe/6843136,2021-10-06 18:23:58
4,5,1,5,밑반찬,아삭아삭한 고추된장박이,풋고추,"된장,다진 마늘,송송썰은 대파,참기름,통깨,매실원액,설탕",https://www.10000recipe.com/recipe/4030952,2021-10-06 18:23:58


In [23]:
#결측치, 행 개수 확인
recipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31961 entries, 0 to 31960
Data columns (total 9 columns):
idx             31961 non-null int64
page_num        31961 non-null int64
seq             31961 non-null int64
cooking_type    31961 non-null object
title           31959 non-null object
ingredient      31412 non-null object
sorce           17739 non-null object
uri             31961 non-null object
insert_dt       31961 non-null object
dtypes: int64(3), object(6)
memory usage: 2.2+ MB


In [24]:
# recipe DataFrame 전처리
recipe_preprocess(recipe)

In [25]:
# 카테고리 입력
cat = get_category()

현재 선택할 수 있는 카테고리는 ['빵', '메인반찬', '찌개', '국 탕', '디저트', '밑반찬', '양식', '퓨전'] 입니다.
찾고 싶은 레시피의 분류를 입력해주세요 : 빵


In [26]:
# 입력받은 메뉴를 str 형태로 반환 (최대 5개까지 입력)
input_recipe = get_ingredients()

최대 5개까지의 재료를 입력해주시고, 입력이 마무리 된 경우 x 입력해주세요
재료를 입력해주세요. : 밀가루
재료를 입력해주세요. : x
선택된 재료들은 ['밀가루'] 입니다.


In [27]:
ref_recipe, origin_recipe = cat_preprocess(cat, recipe)

In [28]:
# 입력받은 레시피 재료와 기존 레시피 재료 DataFrame을 벡터화
recipe_vec = recipe_vectorizer(input_recipe, origin_recipe)

In [29]:
# 벡터화된 레시피 값을 받아 유사도 분석 후 분석된 레시피의 기존 index 값과 유사도 상위 5개를 DataFrame 반환
recipe_index, sim_per = sim_indexPreprocess(recipe_vec)

In [30]:
# 유사도 결과 출력
recommendation_result(recipe_index, ref_recipe)

입력하신 재료와 유사한 레시피 최대 5개를 추출합니다.

레시피 : 손반죽) 쫄깃하고 부드러운 탕종식빵 만들기 (영상/레시피) (유사도:100.0%)
재  료 : 물,밀가루
 url  : https://www.10000recipe.com/recipe/6932249
소  스 : 강력분,소금,설탕,이스트,분유,우유,버터,계란


레시피 : 햄프씨드가 콕콕박힌 흑미식빵~~정말 꼬숩해요 (유사도:85.7%)
재  료 : 밀가루,우유
 url  : https://www.10000recipe.com/recipe/6895333
소  스 : 밀가루장,우유,달걀,제빵용흑미가루,강력분,햄프씨드,설탕,소금,이스트,버터


레시피 : 건강 호박파이 만들기 (노버터, 노오일) (유사도:78.0%)
재  료 : 밀가루,우유,소금
 url  : https://www.10000recipe.com/recipe/6966185
소  스 : 단호박,우유,계란,계피가루,생강가루,후추가루,정향 가루,넛맥가루,설탕,소금


레시피 : 밥솥 우유식빵 (유사도:75.8%)
재  료 : 밀가루,물,우유,설탕,소금,프림,드라이이스트,밀가루,버터,밀가루
 url  : https://www.10000recipe.com/recipe/6940268
소  스 : -


레시피 : [영상] NO오븐 소보로빵 만들기 : '냉장고를 부탁해' 김풍 ‘최후의 소보로’ (유사도:73.9%)
재  료 : 버터,설탕,밀가루,식빵,잼
 url  : https://www.10000recipe.com/recipe/6843635
소  스 : -


